In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pyvi import ViTokenizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string

## Load data

In [4]:
train_df = pd.read_excel(r"C:\Users\MANHLK\Documents\Machine_Learning\Sentiment\Data\train_data_icom_27k.xlsx")

In [5]:
test_df = pd.read_excel(r"C:\Users\MANHLK\Documents\Machine_Learning\Sentiment\Data\test_data_icom_12k.xlsx")

## Data information

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27785 entries, 0 to 27784
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  27785 non-null  int64 
 1   Sentence    27785 non-null  object
 2   Label       27785 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 651.3+ KB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11909 entries, 0 to 11908
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11909 non-null  int64 
 1   Sentence    11909 non-null  object
 2   Label       11909 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 279.2+ KB


In [8]:
count_sentiment = pd.value_counts(train_df['Label'])
count_sentiment

 0    13272
-1     8623
 1     5890
Name: Label, dtype: int64

## Pre-processing data

In [15]:
def pre_processing(text):
    # Chuan hoa tieng viet, tieng anh, thuat ngu
    replace_list={
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
    'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
    'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
    'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
    'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
    #Chuẩn hóa 1 số sentiment words/English words
    ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
    'okey': ' ok ', 'ôkê': ' ok ', ' oki ': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
    ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
    ' not ': u' không ',' khoong ': ' không ', u' kg ': u' không ','ko ': 'không', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' k ':u' không ',u'chẳng':u'không',u'đéo':u'không',
    'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
    ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', ' j ': u' gì ', '“': ' ',
    ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
    'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
    'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
    'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
    'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
    'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
    'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
    'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
    'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
    'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ', ' quả ng ': ' quảng ',
    'dep': u' đẹp ',u'xầu':u'xấu',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u' qủa': u' quả', 
    ' iu ': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
    ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
    'qc':u'quảng cáo',u' éo ':' negative ',' bik ':' biết ',
    #dưới 3* quy về 1*, trên 3* quy về 5*
    '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ', '5 sao': '5star',
    'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ', '3 sao':'5star','4 sao':'5star','1 sao':'1star',
    '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
        
    # Xóa icon
    text = re.sub(r"\W", " ",text)
    
    # Xóa dấu câu
    text = re.sub('['+string.punctuation+']', ' ', text)
    
    # Tokenizer
    text = ViTokenizer.tokenize(text)
    return text

### Pre_processing tran data set

In [21]:
for i in range(train_df.shape[0]):
    train_df['Sentence'][i] = pre_processing(train_df['Sentence'][i])

c:\users\manhlk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Pre_processing test data set

In [22]:
for i in range(test_df.shape[0]):
    test_df['Sentence'][i] = pre_processing(test_df['Sentence'][i])

c:\users\manhlk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Multinomial Naive Bayes Model

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
class NaiveBayesModel:
    def __init__(self):
        self.clf = self._init_pipeline()
    
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf", MultinomialNB()) # navie bayes models
        ])
        return pipe_line

## Support Vector Machine - kernel = 'linear'

In [17]:
from sklearn.svm import SVC

In [19]:
class SVMModel:
    def __init__(self):
        self.clf = self._init_pipeline()
        
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf_svm", SVC(kernel='linear', probability=True)) # svm kernel = 'linear'
        ])
        return pipe_line

## Train model

In [29]:
X_train = train_df['Sentence'].values
y_train = train_df['Label'].values
X_test = test_df['Sentence'].values
y_test = test_df['Label'].values
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(27785,)
(27785,)
(11909,)
(11909,)


### Multinomial Naive Bayes

In [30]:
model = NaiveBayesModel()
clf = model.clf.fit(X_train, y_train)
print("Train accuracy:",clf.score(X_train, y_train))
print("Test accuracy:", clf.score(X_test, y_test))

Train accuracy: 0.7852078459600503
Test accuracy: 0.7408682509026786


### SVM kernel = 'linear'

In [32]:
clf_svm = SVMModel().clf.fit(X_train, y_train)
print("Train accuracy:",clf_svm.score(X_train, y_train))
print("Test accuracy:", clf_svm.score(X_test, y_test))

Train accuracy: 0.8856937196328954
Test accuracy: 0.7952808800067176


### Input from keyboard

In [36]:
test_data = {}
test_data['Sentence']= input('Mời bạn nhập bình luận: ')
test_df = pd.DataFrame(test_data,index=[0])
predict = clf_svm.predict(test_df['Sentence'])[0]
print(predict)

Mời bạn nhập bình luận: tuyệt vời
1
